In [1]:
using Arpack
using LinearAlgebra
using IterativeSolvers
using SparseArrays

In [2]:
function read_mat()
    Is = Int64[]
    Js = Int64[]
    # data downloaded from http://snap.stanford.edu/data/enwiki-2013.html
    open("enwiki-2013.txt") do f
        for line in eachline(f)
            if line[1] == '#'; continue; end # skip lines starting with #
            i, j = split(line)
            # note 0 --> 1 indexing
            push!(Is, parse(Int64, i) + 1)
            push!(Js, parse(Int64, j) + 1)
        end
    end
    n = max(maximum(Is), maximum(Js))
    A = sparse(Is, Js, 1, n, n)
    A = min.(A, 1)
    A = max.(A, A')  # symmetrize
    d = vec(sum(A, dims=1))
    # Skip unused indices
    keep = d .> 0
    return A[keep, keep]
end

read_mat (generic function with 1 method)

In [3]:
@time A = read_mat()  # takes about one minute
d = vec(sum(A, dims=1))
α = 0.5
D = Diagonal(d)
S = D - α * A
;

 77.410068 seconds (405.49 M allocations: 35.570 GiB, 4.76% gc time)


In [4]:
n = size(S, 1)
n, nnz(S)

(4203323, 188082777)

In [5]:
x = randn(n)
@time S * x;

  0.878136 seconds (72.46 k allocations: 35.949 MiB)


In [6]:
# Splitting method for Ax = b
true_x = randn(n)
b = S * true_x
x = zeros(n)
for k = 1:20
    y = α * (A * x) + b
    x = D \ y
    println(norm(x - true_x))
end

303.2860045397605
30.680807329710287
16.33624993932302
2.6898996225196696
2.514796022335498
0.47389121300436904
0.45696159217935234
0.09853462966581672
0.08764067896771156
0.021623018327480426
0.017362415577518626
0.004850734314991793
0.003528499981512804
0.001100288526620871
0.0007336069311031243
0.0002515135310294068
0.00015585963060902707
5.7912848447441255e-5
3.381842567475864e-5
1.343900217793683e-5


In [7]:
@time Λ, Z = eigs(S, tol=1e-2, nev=1, which=:LM)
λ = Λ[1]
z = Z[:, 1]
;

 21.344020 seconds (6.20 M allocations: 1.119 GiB, 0.64% gc time)


In [8]:
norm(S * z - λ * z)

5.855140092601136e-7

In [9]:
# Set us LLS problem
p = 100000
S1 = S[:, 1:p];
y = randn(p)
b = S1 * y
b = b ./ norm(b)
r = rand(n)
r = (r / norm(r)) .* 1e-5
b1 = b + r
;

In [10]:
@time z = lsmr(S1, b1, atol=1e-4, btol=1e-4);

 15.538189 seconds (1.67 M allocations: 362.619 MiB, 1.09% gc time)


In [11]:
norm(S1 * z - b1) / norm(b1)

0.05813522817131246